In [ ]:
# -*- coding: utf-8 -*-
from scipy.cluster import hierarchy
from processor import MDB, batch_process, text_process
from utils import load_obj, dump_obj, progress
from metrics import DTW_hybrid
from pyculiarity import detect_ts
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import pairwise_distances

import re
import math
import datetime
import numpy as np
import fastcluster
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import concurrent.futures 
%matplotlib inline

In [13]:
if not l:
    print 'a'

In [3]:
p = MDB('tweets', 1, port = 27017)

connected to database


In [11]:
l = 1

In [6]:
import re
from utils import progress
from processor import MDB, text_process

f = open('tw_ht_corpus.txt', 'a')
p = MDB('tweets', 1, port = 27017)
cols = p.client['tweets'].collection_names()
cols.remove('SPB')
cols.remove('Moscow')

i = 0
container = []
b_size = 1000

counts = []
for c in cols:
    ml = p.client['tweets'][c].find()
    counts.append(ml.count())
    
total = sum(counts)
print 'total:', total

for c in cols:
    ml = p.client['tweets'][c].find()
    for t in ml:
        try:
            dt = text_process(t)
            progress(i, total)
            if dt:
                f.write(dt + '\n')
        except Exception as e:
            print e
        finally:
            i += 1

connected to database


In [ ]:
for c in cls:
    ml = p.client['tweets'][c].find()
    for t in ml:
        if len(container) < b_size:
            container.append(t)
        else:
            dat = []
            with concurrent.futures.ProcessPoolExecutor(max_workers=1) as executor:
                docs = {executor.submit(text_process, doc) : doc for doc in container}
                for future in concurrent.futures.as_completed(docs):
                    try:
                        data = future.result(timeout = 1)
                        if data:
                            i += 1
                            progress(i, total)
                            dat.append(data + '\n')
                    except Exception as e:
                        print(e)
            #f.writelines(dat)
            container = []

In [ ]:
def retrieve_from_timewindow(ms, col, startEpoch, endEpoch):
    print 'querying..'
    m = ms.db[col].find({"payload.created_at":{'$gte':startEpoch, '$lte':endEpoch}})
    dev = []
    print 'retrieving..'
    for i, obj in enumerate(m):
        progress(i, 1, mode = 2)
        dev.append(obj)
    print ''
    print 'retrieved', len(dev), 'documents from window'
    return dev

In [ ]:
datetime.fromordinal

In [ ]:
def tstamp_to_datetime(ts):
    return pd.to_datetime(datetime.fromtimestamp(int(ts)).strftime('%Y-%m-%d %H:%M:%S'))

def find_window(t, start, window):
    w = math.floor((t-start-0.1)/window)
    if w < 0: w = 0.0
    return w

def movingaverage(interval, window_size):
    window = np.ones(int(window_size))/float(window_size)
    return np.convolve(interval, window, 'same')

def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

def hasBS(inputString):
    if "_" in inputString:
        return 1
    else:
        return 0

In [ ]:
dev = pc
mins = 120

print 'processing', len(dev), 'documents'

t_start = dev[0]['timestamp']
t_end = dev[-1]['timestamp']
t_span = t_end - t_start
resolution = int(math.floor(t_span/(60*mins)))
t_window = t_span/resolution

print 'window size =', mins, 'minutes'

w_corp = [' '.join(m['words']) for m in dev]
CV = CountVectorizer(min_df=3, max_df=len(dev)/100)
wtr = CV.fit_transform(w_corp)
voc = CV.get_feature_names()

print 'vocab size =', len(voc) 

wn = {}

for w in range(resolution):
    wn[w] = {}
    wn[w]['counts'] = np.zeros((1, len(voc)))
    wn[w]['timestamp'] = tstamp_to_datetime(t_start + t_window * w)

for i, d in enumerate(dev):
    w = find_window(d['timestamp'], t_start, t_window)
    wn[w]['counts'] += wtr[i].toarray()
    
nnz = []
for k in wn.keys():
    nonzero = len(wn[k]['counts'].nonzero()[0])
    if nonzero > 0:
        nnz.append(wn[k])

print 'found', len(nnz), 'nonzero windows'

M = []
T = []
for n in nnz:
    M.append(n['counts'][0])
    T.append(n['timestamp'])

M = np.array(M).T

In [ ]:
P = []
ind = []
for term in range(M.shape[0]):
    if M[term].max() > 10 and len(voc[term]) > 3 and not hasNumbers(voc[term]) and not hasBS(voc[term]):
        ind.append(term)
        P.append(M[term])
        
P = np.array(P)
cvoc = [voc[i] for i in ind]

In [ ]:
def estimated_autocorrelation(x):
    n = len(x)
    variance = x.var()
    x = x-x.mean()
    r = np.correlate(x, x, mode = 'full')[-n:]
    assert np.allclose(r, np.array([(x[:n-k]*x[-(n-k):]).sum() for k in range(n)]))
    result = r/(variance*(np.arange(n, 0, -1)))
    return result

In [ ]:
def pr_dist(M):
    dim = M.shape[0]
    R = np.zeros((dim,dim))
    c = 0
    for i in range(dim):
        for j in range(dim):
            progress(i*dim + j, dim*dim, skip = 10)
            if i == j:
                R[i][j] = 0
            elif not R[i][j] and not R[j][i]:
                r = DTWDistance(M[i], M[j])
                c += 1
                R[i][j] = r
                R[j][i] = r
    return R, c

In [ ]:
st = datetime.now()
s_sim = pairwise_distances(P, metric=DTW_hybrid, n_jobs=12)
en = datetime.now()
print str(en-st)

In [ ]:
s_sim = load_obj('data/SPB_DTW')

In [ ]:
cvoc  = load_obj('data/SPB_voc')

In [ ]:
windows = np.array(range(P.shape[1]))
mv_wind_size = int(0.05*len(windows))

for term_n in range(P.shape[0]):
    print 'term: >>>', voc[ind[term_n]]
    plt.plot(windows, P[term_n], 'b-', label='label here')
    plt.plot(windows, movingaverage(P[term_n], mv_wind_size), 'r-', label='label here')
    plt.show()

In [ ]:
for i in range(s_sim.shape[0]):
    t = s_sim[i]
    u = t.argsort()
    if t[u[1]] < 2:
        for j in u[0:5]:
            if t[j] < 2:
                print cvoc[j], t[j],
        print ''
    
    

In [ ]:
def plot(ts, t):
    plt.plot(windows, ts, 'b-', label='label here')
    plt.plot(windows, t * movingaverage(ts, 3), 'r-', label='label here')
    plt.show()

In [ ]:
L = fastcluster.linkage(sm, method='average')
flat_textclust = hierarchy.fcluster(L, 0.5, 'distance')

In [ ]:
react = 'отлично, замечательно'